In [ ]:
import json

data_file = '../game_data.json'


with open(data_file, encoding="utf-8") as f:
    game_data = json.load(f)

In [ ]:
import matplotlib.pyplot as plt

def create_histogram(data, title, xlabel, ylabel, color='tab:blue', figuresize=(10, 6)):
    labels = list(data.keys())
    values = list(data.values())

    n_bars = len(labels)
    figure_width = max(labels) - min(labels)

    bar_width = figure_width / (1.5 * n_bars)

    plt.figure(figsize=figuresize)
    plt.bar(labels, values, color=color, edgecolor='black', width=bar_width)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
# Function to get sentiment scores
weight_neg = 0.7
weight_compound = 1-  weight_neg 

def normalize_score(score, old_min, old_max, new_min, new_max):
    return ((score - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

def get_violence_score(neg_score, pos_score, weight_neg, weight_compound):
    # Normalize 'neg' score from [0, 1] to [-1, 1]
    normalized_neg_score = normalize_score(neg_score, 0, 0.35, 0, 1)
    normalized_pos_score = normalize_score(pos_score, 0, 0.3, 0, 1)
    
    # Calculate the weighted score
    violence_score = normalized_neg_score * weight_neg - normalized_pos_score * weight_compound
    
    return violence_score


def get_sentiment_scores(data, subsections = ["gameplay"]):
    sentiment_scores = {}
    for title, content in data.items():
        text = "\n".join([content['text'][subsection] for subsection in subsections if subsection in content["text"]])
        sentiment = analyzer.polarity_scores(text)
        sentiment["violence"] = get_violence_score(sentiment["neg"], sentiment["pos"], weight_neg, weight_compound)
        sentiment_scores[title] = sentiment
    return sentiment_scores

# Function to create a histogram of sentiment scores
def create_sentiment_histograms(sentiment_data, sentiments_to_plot=None):
    # Default to all sentiment types if none are specified
    if sentiments_to_plot is None:
        sentiments_to_plot = ['pos', 'neu', 'neg', 'compound']

    # Determine the number of plots
    num_plots = len(sentiments_to_plot)
    cols = 2  # We prefer a 2-column layout
    rows = (num_plots + 1) // cols  # Calculate rows needed

    # Setting up the figure for multiple subplots
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    if rows > 1:
        axes = axes.flatten()  # Flatten if we have more than one row
    else:
        axes = [axes]  # Wrap in list if only one row (i.e., 1 or 2 plots)

    fig.suptitle('Sentiment Analysis Histograms')

    # Plotting each requested sentiment
    for i, sentiment in enumerate(sentiments_to_plot):
        scores = [details[sentiment] for details in sentiment_data.values()]
        ax = axes[i]
        ax.hist(scores, bins=200, color='tab:blue', edgecolor='black')
        ax.set_title(f'{sentiment.capitalize()} Sentiment Score')
        ax.set_xlabel('Sentiment Score')
        ax.set_ylabel('Number of Games')

    # Turn off any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    # Adjust layout for better spacing
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

def print_top_bottom_sentiment_games(sentiment_scores, key='compound', n = 10):
    # Sorting the games based on the compound sentiment score
    sorted_games = sorted(sentiment_scores.items(), key=lambda x: x[1][key])
    

    # Printing the 10 most negative games
    print(f"{n} Most Negative Games in terms of {key} sentiment:")
    for game, score in sorted_games[:n]:
        print(f"{game}: {score}")

    print("\n")  # Adding a newline for better readability

    # Printing the 10 most positive games
    print(f"{n} Most Positive Games in terms of {key} sentiment:")
    for game, score in sorted_games[-n:]:
        print(f"{game}: {score}")

def is_subsection_length_valid(data, subsections, min_length, max_length):
    """
    Check if the length of a subsection is within the specified range.
    
    :param data: The data dictionary of a game
    :param subsection: The subsection to check within the data
    :param min_length: The minimum length of the subsection string
    :param max_length: The maximum length of the subsection string
    :return: True if the length is within range, False otherwise
    """
    subsection_text = ""
    for subsection in subsections:
        subsection_text += data.get('text', {}).get(subsection, "") + "\n"
    word_count = len(subsection_text.split())
    return min_length <= word_count <= max_length

def count_long_subsections(game_data, subsection='gameplay', min_length=0, max_length=float('inf')):
    """
    Counts the number of entries in the game_data dictionary that have a specified subsection
    with a string length within the given range.
    
    :param game_data: Dictionary containing game information
    :param subsection: The subsection to look for within the entries (default is 'gameplay')
    :param min_length: The minimum length of the subsection string to count (default is 0)
    :param max_length: The maximum length of the subsection string to count (default is infinity)
    :return: The count of entries with the subsection string length within the specified range
    """
    return sum(is_subsection_length_valid(data, subsection, min_length, max_length) for _, data in game_data.items())

def filter_entries_by_length(game_data, subsections=['gameplay'], min_length=0, max_length=float('inf')):
    """
    Creates a dictionary with only the entries from game_data that have a specified subsection
    with a string length within the given range.
    
    :param game_data: Dictionary containing game information
    :param subsection: The subsection to look for within the entries (default is 'gameplay')
    :param min_length: The minimum length of the subsection string to filter by (default is 0)
    :param max_length: The maximum length of the subsection string to filter by (default is infinity)
    :return: A new dictionary with filtered entries
    """
    return {game: data for game, data in game_data.items() if is_subsection_length_valid(data, subsections, min_length, max_length)}




In [ ]:
from collections import Counter


unique_keys = Counter()
intersection = 0
for game, content in game_data.items():
        intersection += 1 if "plot" in content['text'].keys() and "gameplay" in content['text'].keys() else 0
        unique_keys.update(content['text'].keys())

# Display the unique keys
unique_keys


In [ ]:
import os
import json

def get_or_create_value(function_to_apply, file_path):
    # Check if the file exists
    if os.path.exists(file_path):
        # Load the value from the file
        with open(file_path, 'r') as file:
            value = json.load(file)
    else:
        # Call the function to create the value
        value = function_to_apply()
        # Save the value to the file
        with open(file_path, 'w') as file:
            json.dump(value, file)
    
    return value

# Example usage
# Assuming get_sentiment_scores is a function you have defined elsewhere

In [ ]:
sentiment_scores = get_or_create_value(lambda: {key: get_sentiment_scores(game_data, list(key)) for key in unique_keys.keys()} , 'vader_sentiment_scores.json')


In [ ]:
create_sentiment_histograms(sentiment_scores["plot"])

In [ ]:
create_sentiment_histograms(sentiment_scores["gameplay"])

In [ ]:
create_sentiment_histograms(sentiment_scores["reception"])

In [ ]:
print_top_bottom_sentiment_games(sentiment_scores["plot"])


In [ ]:
sentiment_scores["gameplay"]["Tetris"], sentiment_scores["gameplay"]["God of War (2005 video game)"], sentiment_scores["gameplay"]["Minecraft"], sentiment_scores["gameplay"]["Mortal Kombat (1992 video game)"],

In [ ]:
sorted(sentiment_scores["gameplay"].items(), key=lambda x: -abs(((x[1]["neg"]) * x[1]["compound"])))

In [ ]:
constrains = {
     "min_length": 100,
    "max_length": 10000
}
subsections = ['gameplay','plot','story','synopsis','plot and gameplay','plot and gameplay']

filtered_by_length = filter_entries_by_length(game_data, subsections=subsections, **constrains)

In [ ]:
sentiment_filtered_by_length = get_or_create_value( lambda: get_sentiment_scores(filtered_by_length, subsections), file_path= "filtered_gameplay_sentiment.json")

In [ ]:
# import scipy.stats as stats

# neg_scores = [item['pos']+ 0.00001 for item in sentiment_filtered_by_length.values()]

# # Apply the Box-Cox transformation to the list of 'neg' scores
# # The scipy boxcox function returns the transformed array and the lambda value used
# neg_scores_transformed, lambda_value = stats.boxcox(neg_scores)

# # Update the original dictionary with the transformed 'neg' scores
# for (key, value), transformed_score in zip(sentiment_filtered_by_length.items(), neg_scores_transformed):
#     value['pos'] = transformed_score

In [ ]:
violence = [sentiment["violence"] for game, sentiment in sentiment_filtered_by_length.items()]
for game, sentiment in sentiment_filtered_by_length.items():
    sentiment["violence"] = normalize_score(sentiment["violence"], min(violence), max(violence), 0, 1)

In [ ]:
create_sentiment_histograms(sentiment_data=sentiment_filtered_by_length, sentiments_to_plot= ['pos', 'neu', 'neg', 'compound', "violence"])

In [ ]:
print_top_bottom_sentiment_games(sentiment_filtered_by_length, "compound", n = 50)

In [ ]:
game_data_with_vader_sentiment_on_gameplay = get_or_create_value(lambda: {
    game: {**data, 'sentiment': sentiment_filtered_by_length[game]}
    for game, data in filtered_by_length.items() }, "game_data_with_vader_sentiment.json")

In [ ]:
print_top_bottom_sentiment_games(sentiment_filtered_by_length, "violence", n = 50)

In [ ]:
import pandas as pd

def compare_violence_scores(data, most_violent, least_violent):
    # Initialize dictionaries to hold the scores for the most and least violent games
    most_violences = {'neg': [], 'neu': [], 'pos': [], 'compound': [], "violence": []}
    least_violences = {'neg': [], 'neu': [], 'pos': [], 'compound': [], "violence": []}
    
    # Helper function to calculate average of a list
    def average(lst):
        return sum(lst) / len(lst) if lst else 0
    
    # Extract scores for each game in the most and least violent games lists
    for game in most_violent:
        if game in data:
            most_violences['neg'].append(data[game]['neg'])
            most_violences['neu'].append(data[game]['neu'])
            most_violences['pos'].append(data[game]['pos'])
            most_violences['compound'].append(data[game]['compound'])
            most_violences['violence'].append(data[game]['violence'])
    
    for game in least_violent:
        if game in data:
            least_violences['neg'].append(data[game]['neg'])
            least_violences['neu'].append(data[game]['neu'])
            least_violences['pos'].append(data[game]['pos'])
            least_violences['compound'].append(data[game]['compound'])
            least_violences['violence'].append(data[game]['violence'])
    
    # Calculate averages for both groups
    averages = {
        'Score': ['neg', 'neu', 'pos', 'compound', "violence"],
        'Average Most Violent': [
            average(most_violences['neg']),
            average(most_violences['neu']),
            average(most_violences['pos']),
            average(most_violences['compound']),
            average(most_violences['violence'])
        ],
        'Average Least Violent': [
            average(least_violences['neg']),
            average(least_violences['neu']),
            average(least_violences['pos']),
            average(least_violences['compound']),
            average(least_violences['violence'])
        ]
    }
    
    # Create a DataFrame to display the table
    df = pd.DataFrame(averages)
    
    # Calculate and add a column for the difference between the most and least violent scores
    df['Difference (Most - Least)'] = df['Average Most Violent'] - df['Average Least Violent']
    
    return df

most_violent = ["Doom (2016 video game)", "Grand Theft Auto V", "Mortal Kombat (1992 video game)", "God of War (2005 video game)", "Manhunt (video game)", "Gears of War (video game)", "Call of Duty 4: Modern Warfare", "Dead Space (2008 video game)", "Resident Evil (1996 video game)", "Hotline Miami"]
least_violent = ["Animal Crossing: New Horizons", "Stardew Valley", "The Sims 4", "Minecraft", "Tetris", "Monument Valley (video game)", "Super Mario Odyssey", "Journey (2012 video game)", "Katamari Damacy", "Fez (video game)"]
compare_violence_scores(sentiment_filtered_by_length, most_violent, least_violent)

In [ ]:
def rank_games_by_scores(data, most_violent, least_violent):
    # Flatten the data to a list of tuples for sorting
    # Assuming the correct key is 'neg' not 'violence' based on the provided data
    score = 'violence'
    flattened_data = [(game, scores[score], scores['compound']) for game, scores in data.items()]

    # Initialize a list to hold the rank data
    rank_most = []
    rank_least = []

    # Sort and rank for the most violent games
    for game in most_violent:
        # Lower 'neg' scores and more negative 'compound' scores are ranked higher for most violent games
        neg_rank = sorted(flattened_data, key=lambda x: -x[1]).index((game, data[game][score], data[game]['compound'])) + 1
        compound_rank = sorted(flattened_data, key=lambda x: x[2]).index((game, data[game][score], data[game]['compound'])) + 1
        rank_most.append({'Game': game, 'Violence Category': 'Most Violent', 'Neg Rank': neg_rank, 'neg score': data[game][score], 'Compound Rank': compound_rank, "Compound score":data[game]['compound']})

    # Sort and rank for the least violent games
    for game in least_violent:
        # Higher 'neg' scores and less negative 'compound' scores are ranked higher for least violent games
        neg_rank = sorted(flattened_data, key=lambda x: x[1]).index((game, data[game][score], data[game]['compound'])) + 1
        compound_rank = sorted(flattened_data, key=lambda x: -x[2]).index((game, data[game][score], data[game]['compound'])) + 1
        rank_least.append({'Game': game, 'Violence Category': 'Least Violent', 'Neg Rank': neg_rank, 'neg score': data[game][score], 'Compound Rank': compound_rank, "Compound score":data[game]['compound']})

    # Create the DataFrame from the list of rank data
    ranks_most_df = pd.DataFrame(rank_most)
    ranks_least_df = pd.DataFrame(rank_least)

    return ranks_most_df, ranks_least_df

ranks_most_df, ranks_least_df = rank_games_by_scores(sentiment_filtered_by_length, most_violent, least_violent)


In [ ]:
ranks_most_df

In [ ]:
ranks_least_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to plot score distributions for the most and least violent games.
def plot_score_distributions(data, most_violent, least_violent):
    # Create a DataFrame for the scores
    scores_list = []
    for game in data:
        if game in most_violent or game in least_violent:
            score_data = data[game]
            score_data['Game'] = game
            score_data['Violence'] = 'Most Violent' if game in most_violent else 'Least Violent'
            scores_list.append(score_data)
    
    scores_df = pd.DataFrame(scores_list)

    # Plotting the negativity score distribution
    plt.figure(figsize=(10, 5))
    sns.histplot(data=scores_df, x='violence', hue='Violence', element='step', stat='density', common_norm=False)
    plt.title('Negativity Score Distribution')
    plt.show()

    # Plotting the compound score distribution
    plt.figure(figsize=(10, 5))
    sns.histplot(data=scores_df, x='compound', hue='Violence', element='step', stat='density', common_norm=False)
    plt.title('Compound Score Distribution')
    plt.show()

# Call the function with hypothetical data
plot_score_distributions(sentiment_filtered_by_length, most_violent, least_violent)
